In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

#Keras
from keras.losses import categorical_crossentropy
#from keras.applications.resnet50 import preprocess_input
from keras.utils import Sequence
from keras.utils import to_categorical
from keras.legacy import interfaces
from keras.optimizers import Optimizer
from keras import backend as K

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [3]:
K.backend()

'tensorflow'

In [4]:
df_train = pd.read_csv('/kaggle/input/aptos2019-blindness-detection/train.csv')
df_test = pd.read_csv('/kaggle/input/aptos2019-blindness-detection/test.csv')

In [5]:
df_train.head()

,id_code,diagnosis
0,000c1434d8d7,2
1,001639a390f0,4
2,0024cdab0c1e,1
3,002c21358ce6,0
4,005b95c28852,0


In [6]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

def append_ext(fn):
    return fn+".png"

df_train["id_code"] = df_train["id_code"].apply(append_ext)
df_test["id_code"] = df_test["id_code"].apply(append_ext)
datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [7]:
train_generator=datagen.flow_from_dataframe(
dataframe=df_train,
directory="/kaggle/input/aptos2019-blindness-detection/train_images/",
x_col="id_code",
y_col="diagnosis",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="raw",
target_size=(224,224))

valid_generator=datagen.flow_from_dataframe(
dataframe=df_train,
directory="/kaggle/input/aptos2019-blindness-detection/train_images/",
x_col="id_code",
y_col="diagnosis",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="raw",
target_size=(224,224))

test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
dataframe=df_test,
directory="/kaggle/input/aptos2019-blindness-detection/test_images/",
x_col="id_code",
y_col=None,
batch_size=4,
seed=42,
shuffle=False,
class_mode=None,
target_size=(224,224))

Found 2747 validated image filenames.
Found 915 validated image filenames.
Found 1928 validated image filenames.


In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(224,224,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [9]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20
)

Epoch 1/20
85/85 [==============================] - 375s 4s/step - loss: 1.2210 - acc: 0.6298 - val_loss: 0.9259 - val_acc: 0.6964
Epoch 2/20
85/85 [==============================] - 326s 4s/step - loss: 0.8180 - acc: 0.7151 - val_loss: 0.8870 - val_acc: 0.7067
Epoch 3/20
85/85 [==============================] - 325s 4s/step - loss: 0.7746 - acc: 0.7211 - val_loss: 0.8509 - val_acc: 0.7135
Epoch 4/20
85/85 [==============================] - 323s 4s/step - loss: 0.7615 - acc: 0.7220 - val_loss: 0.8647 - val_acc: 0.6965
Epoch 5/20
85/85 [==============================] - 323s 4s/step - loss: 0.7421 - acc: 0.7370 - val_loss: 0.8227 - val_acc: 0.7044
Epoch 6/20
85/85 [==============================] - 323s 4s/step - loss: 0.7553 - acc: 0.7304 - val_loss: 0.8346 - val_acc: 0.7112
Epoch 7/20
85/85 [==============================] - 322s 4s/step - loss: 0.7297 - acc: 0.7400 - val_loss: 0.8014 - val_acc: 0.7271
Epoch 8/20
85/85 [==============================] - 321s 4s/step - loss: 0.7229 - a

In [11]:
model.evaluate_generator(generator=valid_generator, steps=STEP_SIZE_TEST)

[0.7472246238427114, 0.7257492114171841]

In [12]:
test_generator.reset() # Necessary to force it to start from beginning
pred = model.predict_generator(test_generator, steps=STEP_SIZE_TEST, verbose=1)

482/482 [==============================] - 70s 145ms/step


In [13]:
predicted_class_indices = np.argmax(pred, axis=1)

In [14]:
predicted_class_indices.shape

(1928,)

In [15]:
df_pred = pd.DataFrame(predicted_class_indices)

In [16]:
df_pred[0].value_counts()

2    1506
0     215
1     177
3      17
4      13
Name: 0, dtype: int64

In [17]:
len(test_generator.filenames)

1928

In [18]:
y_pred = predicted_class_indices
# Create dataframe for submission
df_submit = pd.DataFrame({'id_code':test_generator.filenames, 'diagnosis':y_pred})
# Filename is id, remove extension .png
df_submit['id_code'] = df_submit['id_code'].apply(lambda x: x.split('.')[0])
print(f"Submission dataframe created. Rows:{len(df_submit.values)}")

Submission dataframe created. Rows:1928


In [21]:
# Write to csv
df_submit.to_csv('submission.csv', index=False)

In [20]:
df_submit.head()

,id_code,diagnosis
0,0005cfc8afb6,2
1,003f0afdcd15,2
2,006efc72b638,2
3,00836aaacf06,2
4,009245722fa4,2
